In [29]:
import numpy as np
import json
import warnings

import pandas as pd
import tensorflow as tf
from tensorflow.keras import models, Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, BatchNormalization, Dropout, Input, Concatenate, GlobalAvgPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
#from tensorflow.keras.callbacks import ReduceLROnPlateau
import json
from evaluator import *

In [30]:
VARIABLES = ['ActivityCounts', 'Barometer', 'BloodPerfusion',
             'BloodPulseWave', 'EnergyExpenditure', 'GalvanicSkinResponse', 'HR',
             'HRV', 'RESP', 'Steps', 'SkinTemperature', 'ActivityClass']

In [31]:
GRAYSCALE = False # grayscale or rgb # TODO: currently not working (MobileNetV2 does not support grayscale)
TRANSFORMER_IMPUTATION = True # whether to use transformer imputation or linear interpolation
FULL_TRANSFORMER_IMPUTATION = False # whether to use FULL transformer imputation or linear interpolation
BACKGROUND_FEATURES = False

# Import data

In [33]:
# file path to data folder
path = './Output' if not TRANSFORMER_IMPUTATION else './Output/transformer_imputation'
path = './Output/full_transformer_imputation' if FULL_TRANSFORMER_IMPUTATION else path
path

'./Output/transformer_imputation'

In [34]:
# dimensions
N, HEIGHT, WIDTH, CHANNELS = sum([1 for p in os.listdir(path) if (p[:14] == 'feature_vector' and p[:19] != 'feature_vector_stat')]), \
                             *np.load(path + '/feature_vector0.npy').shape
CHANNELS = len(VARIABLES) if GRAYSCALE else CHANNELS # reduce channels for grayscale

print(N, HEIGHT, WIDTH, CHANNELS)

613 370 497 30


Metadata (subjectID etc.)

In [35]:
with open(path + '/metadata.txt') as f:
    metadata = f.read()

metadata = json.loads(metadata.replace('\'', '\"').replace('False', 'false').replace('True', 'true')) # doesn't accept other chars

In [36]:
subjects = [meta['subjectID'] for meta in metadata]

In [37]:
# age: 26-35: [0, 0], 36-45: [0, 1], 46-55: [1, 0] (subject 22: unknown -> randomly assigned)
age_ranges = [np.array([0, 0]), np.array([0, 1]), np.array([1, 0])]
age = {1: age_ranges[1],
       2: age_ranges[1],
       3: age_ranges[1],
       4: age_ranges[1],
       5: age_ranges[1],
       6: age_ranges[2],
       7: age_ranges[1],
       8: age_ranges[2],
       9: age_ranges[1],
       10: age_ranges[1],
       11: age_ranges[1],
       12: age_ranges[2],
       13: age_ranges[2],
       14: age_ranges[1],
       15: age_ranges[2],
       16: age_ranges[0],
       17: age_ranges[2],
       18: age_ranges[2],
       19: age_ranges[1],
       20: age_ranges[2],
       21: age_ranges[2],
       22: age_ranges[np.random.choice((0, 1, 2))],
       23: age_ranges[0],
       24: age_ranges[0],
       25: age_ranges[2],
       26: age_ranges[0],
       27: age_ranges[1],
       28: age_ranges[0]}
age = [age[subject] for subject in subjects]

In [38]:
# gender: 0: male, 1: female (subject 22: unknown -> randomly assigned)
gender = {1: 0,
          2: 0,
          3: 0,
          4: 1,
          5: 1,
          6: 1,
          7: 0,
          8: 0,
          9: 0,
          10: 1,
          11: 1,
          12: 1,
          13: 0,
          14: 1,
          15: 1,
          16: 1,
          17: 0,
          18: 0,
          19: 0,
          20: 0,
          21: 0,
          22: np.random.choice((0, 1)),
          23: 1,
          24: 0,
          25: 0,
          26: 0,
          27: 1,
          28: 0}
gender = [gender[subject] for subject in subjects]

# CNN

Addditional functions

In [39]:
# image-wise transformer
def rgb2gray(rgb):
    """greyscale = 0.2989 * red + 0.5870 * green + 0.1140 * blue"""
    return np.dot(rgb[:, :, :3], [0.2989, 0.5870, 0.1140])

# loss function
def weighted_cross_entropy(weight):
    def weighted_cross_entropy_with_logits(labels, logits):
        loss = tf.nn.weighted_cross_entropy_with_logits(
            labels, logits, weight
        )
        return loss
    return weighted_cross_entropy_with_logits

# weight (imbalanced classes)
def check_imbalance(path_to_labels, indices, variable):
    """Returns indices of positives/negatives"""
    y = np.empty((len(indices), 2), dtype=int)
    for i, index in enumerate(indices):
        y[i, ] = np.load(path_to_labels + f'/labels{index}.npy', allow_pickle=True)

    positives = np.where(y[:, variable] == 1)[0]
    negatives = np.where(y[:, variable] == 0)[0]

    return np.array(indices)[positives], np.array(indices)[negatives]

def get_weighting_factor(path, train_set_indices, variable):
    positives, negatives = check_imbalance(path, train_set_indices, variable)
    sample_weight = len(negatives) / len(positives) # for weighted cross-entropy
    return sample_weight

Dataloader (dataset with images too large)

In [40]:
class DataGenerator(Sequence):

    def __init__(self, data_path: str, indices_dataset: list, variable, batch_size=32, dim=(HEIGHT, WIDTH), n_channels=CHANNELS, shuffle=True):
        self.data_path = data_path # path to full dataset
        self.dim = dim # image dimension
        self.batch_size = batch_size
        self.indices_dataset = indices_dataset # indices of full dataset (different for train/validation/test set)
        self.n_channels = n_channels
        self.shuffle = shuffle
        assert variable in (0, 1)
        self.variable = variable

        self.on_epoch_end() # shuffle data for each epoch

    def on_epoch_end(self):
        """
        Shuffle data for each epoch
        """
        if self.shuffle:
            np.random.shuffle(self.indices_dataset)

    def __data_generation(self, indices):
        """
        Loads and returns datapoints[indices]
        """
        # init
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty(self.batch_size, dtype=float) # float: logits, int: non-logits

        # load individual datapoints
        for i, index in enumerate(indices):
            images = np.load(self.data_path + f'/feature_vector{index}.npy', allow_pickle=True)
            if GRAYSCALE:
                images_gray = np.empty((HEIGHT, WIDTH, self.n_channels))
                for j in range(len(VARIABLES)):
                    image_rgb = images[:, :, (3 * j): (3 * (j + 1))]
                    image_gray = rgb2gray(image_rgb)
                    images_gray[:, :, j] = image_gray
                images = images_gray

            X[i, ] = images
            y[i] = np.load(self.data_path + f'/labels{index}.npy', allow_pickle=True)[self.variable]

        if BACKGROUND_FEATURES:
            global age, gender
            background_features = np.empty((self.batch_size, *self.dim, 3)) # create pseudo-image to comply with network
            for i, index in enumerate(indices):
                age_i, gender_i = age[i], gender[i]
                background_features[i, ] = np.dstack([np.ones(self.dim) * age_i[0],
                                                     np.ones(self.dim) * age_i[1],
                                                     np.ones(self.dim) * gender_i])

            X = np.concatenate([X, background_features], axis=-1)

        return X, y

    def __len__(self):
        """
        Number of batches per epoch
        """
        return int(np.floor(len(self.indices_dataset) / self.batch_size))

    def __getitem__(self, index):
        """
        Generates batch[index]
        """
        # calculate indices of batch
        indices = self.indices_dataset[index * self.batch_size:(index + 1) * self.batch_size]

        # generate batch
        X, y = self.__data_generation(indices)

        return X, y

Architecture

In [41]:
class ConvNet_larger(tf.keras.Model):

    def __init__(self, name='cnn', **kwargs):
        super(ConvNet_larger, self).__init__(name, **kwargs)

        self.in_shape = (HEIGHT, WIDTH, CHANNELS) if not BACKGROUND_FEATURES else (HEIGHT, WIDTH, CHANNELS + 3)
        self.in_shape_mobilenet = (HEIGHT, WIDTH, 3) if not GRAYSCALE else (HEIGHT, WIDTH, 1)

        # MobileNetV2 embedding
        self.mobilenet = MobileNetV2(input_shape=self.in_shape_mobilenet, weights='imagenet', include_top=False)
        self.mobilenet._name = 'mobilenet'
        self.mobilenet.trainable = False
        self.finetuning = False
        self.out_shape_mobilenet = self.mobilenet.layers[-1].output_shape # for one spectrogram

        # Concatenation
        self.concat = Concatenate(name='concat')

        # Global pooling
        '''self.pool = GlobalAvgPool2D(name='global_avg_pool')'''

        # TODO: more sophisticated dense (dropout, regularizer, init., ...)
        self.conv0 = Conv2D(filters=4, kernel_size=(1, 1), activation='relu', padding='same', name='conv0')
        self.conv1 = Conv2D(filters=8, kernel_size=(3, 3), activation='relu', padding='same', name='conv1')
        self.batch_norm1 = BatchNormalization(name='batch_norm1') # try for incr. stability
        self.pool1 = MaxPool2D(strides=(2, 2), name='pool1')

        self.conv2 = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same', name='conv2')
        self.batch_norm2 = BatchNormalization(name='batch_norm2') # try for incr. stability
        self.pool2 = MaxPool2D(strides=(2, 2), name='pool2')

        self.flatten = Flatten(name='flatten')

        self.dense = Dense(10, name='dense')

        # Fully-connected network
        if BACKGROUND_FEATURES:
            self.concat2 = Concatenate(name='concat2', axis=-1)
        self.dense2 = Dense(1, name='dense2') # keep logits
        self.out_shape = 1

        # build graph
        self.build_graph()

    def build_graph(self):
        self.build(input_shape=(None, *self.in_shape))
        x = Input(shape=self.in_shape)
        Model(inputs=[x], outputs=self.call(x))

    def set_finetuning(self, mode=True):
        self.finetuning = mode
        self.mobilenet.trainable = mode

        for layers in self.mobilenet.layers:
            layers.trainable = False

        # "activate" last conv layer of MobileNet
        self.mobilenet.layers[-3].trainable = mode
        self.mobilenet.layers[-2].trainable = mode

    def call(self, inputs, background_features=None):
        """
        Model predictions (logits)
        :param background_features: additional features of shape (3)
        :param inputs: all spectrograms of shape (HEIGHT, WIDTH, CHANNELS)
        :return: class prediction (logits)
        """
        if BACKGROUND_FEATURES:
            inputs = inputs[:, :, :, :-3]
            background_features = inputs[:, 0, 0, -3:]

        # MobileNetV2 embeddings
        x = [self.mobilenet(inputs[..., i:i+3], training=self.finetuning) for i in range(0, CHANNELS, 3)]

        # Concatenation
        x = self.concat(x)

        # Global pooling
        '''x = self.pool(x)'''
        x= self.conv0(x)
        x = self.conv1(x)
        x = self.batch_norm1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.batch_norm2(x)
        x = self.pool2(x)
        x = self.flatten(x)

        if background_features is not None:
            x = self.concat2([x, background_features])

        # Fully-connected network
        x = self.dense(x)
        x = self.dense2(x)

        return x

In [42]:
class ConvNet(tf.keras.Model):

    def __init__(self, name='cnn', **kwargs):
        super(ConvNet, self).__init__(name, **kwargs)

        self.in_shape = (HEIGHT, WIDTH, CHANNELS) if not BACKGROUND_FEATURES else (HEIGHT, WIDTH, CHANNELS + 3)
        self.in_shape_mobilenet = (HEIGHT, WIDTH, 3) if not GRAYSCALE else (HEIGHT, WIDTH, 1)

        # MobileNetV2 embedding
        self.mobilenet = MobileNetV2(input_shape=self.in_shape_mobilenet, weights='imagenet', include_top=False)
        self.mobilenet._name = 'mobilenet'
        self.mobilenet.trainable = False
        self.finetuning = False
        self.out_shape_mobilenet = self.mobilenet.layers[-1].output_shape # for one spectrogram

        # Concatenation
        self.concat = Concatenate(name='concat')

        # Global pooling
        self.pool = GlobalAvgPool2D(name='global_avg_pool')

        # TODO: more sophisticated dense (dropout, regularizer, init., ...)
        # Fully-connected network
        if BACKGROUND_FEATURES:
            self.concat2 = Concatenate(name='concat2', axis=-1)
        self.dense = Dense(1, name='dense') # keep logits
        self.out_shape = 1

        # build graph
        self.build_graph()

    def build_graph(self):
        self.build(input_shape=(None, *self.in_shape))
        x = Input(shape=self.in_shape)
        Model(inputs=[x], outputs=self.call(x))

    def set_finetuning(self, mode=True):
        self.finetuning = mode
        self.mobilenet.trainable = mode

        for layers in self.mobilenet.layers:
            layers.trainable = False

        # "activate" last conv layer of MobileNet
        self.mobilenet.layers[-3].trainable = mode
        self.mobilenet.layers[-2].trainable = mode

    def call(self, inputs, background_features=None):
        """
        Model predictions (logits)
        :param background_features: additional features of shape (3)
        :param inputs: all spectrograms of shape (HEIGHT, WIDTH, CHANNELS)
        :return: class prediction (logits)
        """
        if BACKGROUND_FEATURES:
            inputs = inputs[:, :, :, :-3]
            background_features = inputs[:, 0, 0, -3:]

        # MobileNetV2 embeddings
        x = [self.mobilenet(inputs[..., i:i+3], training=self.finetuning) for i in range(0, CHANNELS, 3)]

        # Concatenation
        x = self.concat(x)

        # Global pooling
        x = self.pool(x)

        if background_features is not None:
            x = self.concat2([x, background_features])

        # Fully-connected network
        x = self.dense(x)

        return x

Model

In [43]:
class CNN:

    def __init__(self, path, variable, epochs, learning_rate, batch_size):
        self.model = ConvNet()
        self.path = path
        assert variable in (0, 1)
        self.variable = variable
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.history = None
        self.train_indices = None
        self.test_indices = None

    def fit(self, train_indices, validation_indices=None):
        self.train_indices = train_indices

        # training set
        train_dataloader = DataGenerator(self.path, train_indices, batch_size=self.batch_size, variable=self.variable)

        # validation set
        if validation_indices is not None:
            val_dataloader = DataGenerator(self.path, validation_indices, batch_size=self.batch_size, variable=self.variable)

        # weights for loss function
        sample_weights = get_weighting_factor(self.path, train_indices, self.variable)

        # build model
        self.model.compile(optimizer=Adam(learning_rate=self.learning_rate),
                           loss=weighted_cross_entropy(sample_weights))

        # training
        if validation_indices is not None:
            self.history = self.model.fit_generator(generator=train_dataloader,
                                                    validation_data=val_dataloader,
                                                    epochs=self.epochs).history
        else:
            self.history = self.model.fit_generator(generator=train_dataloader,
                                                    epochs=self.epochs).history

    def reset(self):
        """Resets model weights"""
        self.model = ConvNet()

    def predict(self, test_indices, logits=False):
        """Predicts actual class labels by default (not logits/probability values)"""
        self.test_indices = test_indices

        # TODO: make more efficient
        # test set + predict
        y_pred = np.empty(len(test_indices), dtype=float)

        for i, index in enumerate(test_indices):
            X_i = np.load(path + f'/feature_vector{index}.npy', allow_pickle=True)
            X_i = tf.expand_dims(X_i, axis=0) # add "batch dimension"

            if BACKGROUND_FEATURES:
                global age, gender
                age_i, gender_i = age[index], gender[index]
                background_features = np.dstack([np.ones((X_i.shape[1], X_i.shape[2])) * age_i[0],
                                                 np.ones((X_i.shape[1], X_i.shape[2])) * age_i[1],
                                                 np.ones((X_i.shape[1], X_i.shape[2])) * gender_i])
                background_features = tf.expand_dims(background_features, axis=0) # add "batch dimension"

                X_i = np.concatenate([X_i, background_features], axis=-1)

            logits_pred_i = self.model.predict(X_i)

            y_pred[i] = logits_pred_i

        if logits:
            return y_pred

        y_probs = tf.math.sigmoid(y_pred) # logits to probs
        y_pred = tf.round(y_probs) # probs to labels

        return y_pred

    def summary(self):
        return self.model.summary()

In [44]:
CNN(path, variable=0, epochs=10, learning_rate=1e-3, batch_size=16).summary()

Model: "conv_net_81"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet (Functional)       (None, 12, 16, 1280)      2257984   
_________________________________________________________________
concat (Concatenate)         (None, 12, 16, 12800)     0         
_________________________________________________________________
global_avg_pool (GlobalAvera (None, 12800)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 12801     
Total params: 2,270,785
Trainable params: 12,801
Non-trainable params: 2,257,984
_________________________________________________________________


# CV

In [45]:
%%time
scores_strat_group_k_fold = [None]*2
scores_loso = [None]*2
scores_strat_k_fold = [None]*2

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')

    for variable in (0, 1): # phF, MF
        model = CNN(path, variable=variable, epochs=10, learning_rate=1e-3, batch_size=16)

        scores_strat_group_k_fold[variable] = stratified_group_k_fold(path=path,
                                                            groups=subjects,
                                                            model=model,
                                                            folds=5,
                                                            images=True,
                                                            verbose=True,
                                                            variable=variable)

        scores_loso[variable] = leave_one_subject_out(path=path,
                                            groups=subjects,
                                            model=model,
                                            images=True,
                                            verbose=True,
                                            variable=variable)

        scores_strat_k_fold[variable] = stratified_k_fold(path=path,
                                                          model=model,
                                                          folds=5,
                                                          images=True,
                                                          verbose=True,
                                                          variable=variable)

Starting stratified group 5-fold for physical fatigue


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/10
36/36 [==============================] - 73s 1s/step - loss: 0.5009
Epoch 2/10
36/36 [==============================] - 24s 644ms/step - loss: 0.3349
Epoch 3/10
36/36 [==============================] - 24s 643ms/step - loss: 0.3002
Epoch 4/10
36/36 [==============================] - 24s 644ms/step - loss: 0.2833
Epoch 5/10
36/36 [==============================] - 24s 643ms/step - loss: 0.2754
Epoch 6/10
36/36 [==============================] - 24s 643ms/step - loss: 0.2541
Epoch 7/10
36/36 [==============================] - 24s 670ms/step - loss: 0.2329
Epoch 8/10
36/36 [==============================] - 23s 640ms/step - loss: 0.2273
Epoch 9/10
36/36 [==============================] - 23s 641ms/step - loss: 0.2093
Epoch 10/10
36/36 [==============================] - 23s 640ms/step - loss: 0.2111


 Fold 1 F1: 0.8148148148148148:  20%|██        | 1/5 [05:09<20:39, 309.95s/it]

Epoch 1/10
24/24 [==============================] - 40s 640ms/step - loss: 0.5588
Epoch 2/10
24/24 [==============================] - 16s 638ms/step - loss: 0.4697
Epoch 3/10
24/24 [==============================] - 16s 638ms/step - loss: 0.3709
Epoch 4/10
24/24 [==============================] - 16s 639ms/step - loss: 0.2781
Epoch 5/10
24/24 [==============================] - 16s 639ms/step - loss: 0.2627
Epoch 6/10
24/24 [==============================] - 16s 638ms/step - loss: 0.2699
Epoch 7/10
24/24 [==============================] - 16s 639ms/step - loss: 0.2347
Epoch 8/10
24/24 [==============================] - 16s 641ms/step - loss: 0.2252
Epoch 9/10
24/24 [==============================] - 16s 642ms/step - loss: 0.2164
Epoch 10/10
24/24 [==============================] - 16s 640ms/step - loss: 0.2311


...

Epoch 1/10
30/30 [==============================] - 43s 632ms/step - loss: 0.7618
Epoch 2/10
30/30 [==============================] - 19s 635ms/step - loss: 0.4495
Epoch 3/10
30/30 [==============================] - 19s 635ms/step - loss: 0.4696
Epoch 4/10
30/30 [==============================] - 19s 635ms/step - loss: 0.4488
Epoch 5/10
30/30 [==============================] - 19s 635ms/step - loss: 0.3908
Epoch 6/10
30/30 [==============================] - 19s 634ms/step - loss: 0.4061
Epoch 7/10
30/30 [==============================] - 19s 634ms/step - loss: 0.3811
Epoch 8/10
30/30 [==============================] - 19s 635ms/step - loss: 0.3237
Epoch 9/10
30/30 [==============================] - 19s 633ms/step - loss: 0.3650
Epoch 10/10
30/30 [==============================] - 19s 634ms/step - loss: 0.3047


 Fold 4 F1: 0.6727726750663083:  80%|████████  | 4/5 [16:48<04:11, 251.89s/it]

Epoch 1/10
30/30 [==============================] - 44s 625ms/step - loss: 0.6075
Epoch 2/10
30/30 [==============================] - 19s 625ms/step - loss: 0.4564
Epoch 3/10
30/30 [==============================] - 19s 625ms/step - loss: 0.5315
Epoch 4/10
30/30 [==============================] - 19s 624ms/step - loss: 0.4404
Epoch 5/10
30/30 [==============================] - 19s 625ms/step - loss: 0.4280
Epoch 6/10
30/30 [==============================] - 19s 625ms/step - loss: 0.3831
Epoch 7/10
30/30 [==============================] - 19s 625ms/step - loss: 0.4317
Epoch 8/10
30/30 [==============================] - 19s 625ms/step - loss: 0.3522
Epoch 9/10
30/30 [==============================] - 19s 625ms/step - loss: 0.3508
Epoch 10/10
30/30 [==============================] - 19s 625ms/step - loss: 0.3303


 Fold 5 F1: 0.8152532135174373: 100%|██████████| 5/5 [20:56<00:00, 251.38s/it]

Performance model:
 accuracy: 0.654 +- 0.126 

 balanced_accuracy: 0.673 +- 0.084 

 f1: 0.644 +- 0.146 

 recall: 0.654 +- 0.126 

 precision: 0.727 +- 0.053 

CPU times: total: 6h 14min 57s
Wall time: 5h 37min 42s


# Save scores

In [50]:
path_scores = './Scores'
model_name = 'cnn'

In [51]:
# stratified group 5-fold
with open(f'{path_scores}/strat_group_5_fold//{model_name}.txt', 'w') as dat:
    dat.write(str(scores_strat_group_k_fold))

In [52]:
# stratified 5-fold
with open(f'{path_scores}/strat_5_fold/{model_name}.txt', 'w') as dat:
    dat.write(str(scores_strat_group_k_fold))

In [53]:
# LOSO
with open(f'{path_scores}/loso//{model_name}.txt', 'w') as dat:
    dat.write(str(scores_loso))